<a href="https://colab.research.google.com/github/Uriel1201/HelloSQL2.0/blob/main/14_cummulative/query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#14. Cumulative salary sum
**Writing a query to get, for each month, the cumulative sum of an employee’s salary excluding the
most recent month.**

In [ ]:
!pip install polars
import pandas as pd
import numpy as  np
import polars as pl

In [ ]:
data = {'id'        : [1,2,1,2,3,1,3,1,3],
        'pay_month' : [1,1,2,2,2,3,3,4,4],
        'salary'    : [20,20,30,30,40,40,60,60,70]
        }

pd_employee=pd.DataFrame(data)
employee=pl.LazyFrame(data)

In [ ]:
print(f'----PANDAS----\nemployee table:\n{pd_employee}')

----PANDAS----
employee table:
   id  pay_month  salary
0   1          1      20
1   2          1      20
2   1          2      30
3   2          2      30
4   3          2      40
5   1          3      40
6   3          3      60
7   1          4      60
8   3          4      70


In [ ]:
pd_df1=pd_employee.copy()
pd_df1['cummulative_pay']=pd_df1.groupby('id')['salary'].cumsum()
pd_df1['ranking']=pd_df1.groupby('id')['pay_month'].rank(method='first',ascending=False)
pd_query=pd_df1.query("ranking>1")[['id','cummulative_pay']]
print(f'{pd_query}')

   id  cummulative_pay
0   1               20
1   2               20
2   1               50
4   3               40
5   1               90
6   3              100


In [ ]:
sample=employee.head(5)
print(f'----POLARS----\nemployee table (SAMPLE)\n{sample.collect()}')

----POLARS----
employee table (SAMPLE)
shape: (5, 3)
┌─────┬───────────┬────────┐
│ id  ┆ pay_month ┆ salary │
│ --- ┆ ---       ┆ ---    │
│ i64 ┆ i64       ┆ i64    │
╞═════╪═══════════╪════════╡
│ 1   ┆ 1         ┆ 20     │
│ 2   ┆ 1         ┆ 20     │
│ 1   ┆ 2         ┆ 30     │
│ 2   ┆ 2         ┆ 30     │
│ 3   ┆ 2         ┆ 40     │
└─────┴───────────┴────────┘


In [ ]:
cumulative=(employee.sort(by=['id','pay_month'])
                    .with_columns(cumulative_pay=pl.col('salary')
                                                   .cum_sum()
                                                   .over('id'),
                                  rank=pl.col('pay_month')
                                         .rank(method='dense',
                                               descending=True
                                          )
                                         .over('id')
                     )
                    .filter(pl.col('rank')>1)
                    .select(pl.col('id'),
                            pl.col('pay_month'),
                            pl.col('cumulative_pay')
                     )
)
print(f'Returning the cumulative pays excluding the last month:\n {cumulative.collect()}')

Returning the cumulative pays excluding the last month:
 shape: (6, 3)
┌─────┬───────────┬────────────────┐
│ id  ┆ pay_month ┆ cumulative_pay │
│ --- ┆ ---       ┆ ---            │
│ i64 ┆ i64       ┆ i64            │
╞═════╪═══════════╪════════════════╡
│ 1   ┆ 1         ┆ 20             │
│ 1   ┆ 2         ┆ 50             │
│ 1   ┆ 3         ┆ 90             │
│ 2   ┆ 1         ┆ 20             │
│ 3   ┆ 2         ┆ 40             │
│ 3   ┆ 3         ┆ 100            │
└─────┴───────────┴────────────────┘
